In [1]:
#import packages
import csv
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
from joblib import dump, load

In [9]:
#return class amount
def getClassAmount(centroids):
    return len(centroids)

In [2]:
#load centroids from step2 clustering results
def getCentroids():
    cs = []
    with open("centroids.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            drow = []
            for col in row:
                drow.append(float(col))
            cs.append(drow)
    #cs = np.array(cs)
    return cs

In [3]:
#每個sample使用features如下，共11個。
#['danceability','energy','key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness',
# 'liveness','valence','tempo']
#classify a feature vector. x must be a 1x11 vector(11 features)
def classify(centroids,x):
    minDis = 0
    belong = 0
    for i in range(len(centroids)):
        val = 0
        for j in range(len(centroids[0])):
            val += math.pow(x[j]-centroids[i][j],2)
        if i == 0:
            minDis = val
        elif val < minDis:
            minDis = val
            belong = i
    return belong

In [4]:
#load pca model from step2
pca = load("pca.joblib")

In [5]:
#get centroids
centroids = getCentroids()

In [10]:
#get class amount
classAmount = getClassAmount(centroids)

In [11]:
#load data
data = []
header = []
if True:
    with open("city_result_2.csv", "r") as csvfile:
        rows = csv.reader(csvfile)
        i = 0
        for row in rows:
            if i == 0:
                header = row[2:13]
            else:
                data.append(row[2:13])
            i += 1

In [12]:
#transform data with pca model
data = np.array(data, dtype=float)
data = pca.transform(data)

In [13]:
#classify every sample in data and output nums of each class
nums = [0]*classAmount
for i in range(len(data)):
    nums[classify(centroids, data[i])]+=1
print(nums)

[273, 274, 525, 437, 491]
